# 1. Obtaining Images & Stamps
For the Rubin Science Platform at <a href="https://data.lsst.cloud">data.lsst.cloud</a><br>
Data Release: <a href="https://dp1.lsst.io/">Data Preview 1 (DP1)</a> <br>

*This notebook is the first in a series of notebooks that cumulatively outline the entire SL-AGN injection and difference image analysis pipeline.*

**Learning Objective:** To learn how to query DP1 for different image types using Butler and generate rotated stamps for source injection

**LSST Data Products:** `visit_image`, `deep_coadd`

**Packages:** `lsst.daf.butler`, `lsst.rsp`, `lsst.afw.display`, `lsst.source.injection`

**Credit:** This notebook was created by Dhruv Sharma with additional code developed by Shenming Fu (SLAC National Accelerator Laboratory)

## 0. Import packages

Start by importing the various files in `lib` to access their helpful supplementary methods. We will also use the `butler` module of the `lsst.daf` package, as well as `lsst.afw`'s `display` module to view our images.

In [ ]:
from lsst.daf.butler import Butler, Timespan
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display as afwDisplay
from lib.stamp import *
from lib.tools import *

## 1. Obtaining Images from Butler

### 1.1. Using the Butler to retrieve visit images

Create a new instance of the Butler configured to query DP1 data from the LSSTComCam.

In [ ]:
BUTLER_CONFIG = "dp1"
COLLECTIONS = "LSSTComCam/DP1"
butler = Butler(BUTLER_CONFIG, collections=COLLECTIONS) 
afwDisplay.setDefaultBackend('matplotlib')
%matplotlib inline

Use the Butler's `query_datasets` task to find `visit_image`s corresponding to the  band and ra,dec (right-ascension, declination) coordinates given in the "bind" of the formatted string, in this case:
>ra = 37.93, dec = 6.93 <br>
>band = 'r'

Typically there will be multiple visit images covering the same ra,dec position, as seen by printing the length of `visit_data`.

In [ ]:
band = 'r'
ra = 37.93
dec = 6.93
query = "band.name = :band AND " \
        "visit_detector_region.region OVERLAPS POINT(:ra, :dec)"
bind_params = {"band": band, "ra": ra, "dec": dec}
visit_data = butler.query_datasets("visit_image", 
                                   where=query,
                                   bind=bind_params,
                                   order_by=["visit.timespan.begin"])
len(visit_data)

Now that we have a list of `visit_image`s containing our desired ra,dec coordinate, let's use the `butler.get` method to obtain 5 visits that we'll use for our stamp injection.

In [ ]:
visit_0 = butler.get(visit_data[0])
visit_1 = butler.get(visit_data[2])
visit_2 = butler.get(visit_data[4])
visit_3 = butler.get(visit_data[6])
visit_4 = butler.get(visit_data[8])

### 1.2. Using the Butler to retrieve coadd images

Let's also fetch the `deep_coadd` image that covers the ra,dec coordinate we queried for, this time using the band name and patch of sky containing our ra,dec.

In [ ]:
query = "band.name = :band AND " \
        "patch.region OVERLAPS POINT(:ra, :dec)"
coadd_data = butler.query_datasets("deep_coadd", 
                                   where=query,
                                   bind=bind_params)
coadd = butler.get(coadd_data[0])

## 2. Stamps

Injecting stamps (in the form of `.fits` files) enables us to insert many different kinds of sources beyond the standard `Star` and `Sersic` models already provided in our catalog generation methods. However, stamps are always injected the same way in all images, irrespective of what orientation that image has.

This means that injecting into `deep_coadd` and `visit_image` data must be a considered process, as we know `deep_coadd`s cover a single patch and are provided to us aligned to the ra,dec coordinate system, whereas `visit_image`s may be rotated by a given boresight angle (counterclockwise from +y). Therefore, if we want our stamps to remain consistent in their orientation between both products, we must rotate stamps injected into a `visit_image` relative to the `deep_coadd`.

### 2.1. Stamp Rotations

Luckily for us, `lib/stamp.py` defines a method `make_rotated_stamp` that accepts a `visit_image` and a `.fits` stamp filename as parameters, and saves a new version of the stamp with its own wcs, as well as one rotated to match the `visit_image`'s wcs to the `fig/` folder as `rotated_system_A_B_wcs.fits`.

Currently our `fig/` folder contains many stamps with names formatted as `system_A_B`, where A denotes which SL-AGN system the stamp represents and B is a time index, letting us represent the variations in appearance an SL-AGN system exhibits as the lensed image magnitudes change over time.

We have 4 systems (1, 2, 4, and 5), each simulated at 5 different times for a total of 20 stamps. We want each `visit_image` to contain many injected instances of the same stamp system at the same time. Then for every other visit, we'll change which time index we use to simulate observing the same SL-AGN system at multiple times over many captures. So we'll inject copies of `visit_0` with `system_A_0.fits` for each system A, and each `visit_1` we inject will use `system_A_1.fits`, and so on for each visit and each SL-AGN system.

Run the below code to generate the 20 rotated stamps.

In [ ]:
for i in ([1,2,4,5]):
    make_rotated_stamp(visit_0, 'fig/system_%s_0.fits'%i)
    make_rotated_stamp(visit_1, 'fig/system_%s_1.fits'%i)
    make_rotated_stamp(visit_2, 'fig/system_%s_2.fits'%i)
    make_rotated_stamp(visit_3, 'fig/system_%s_3.fits'%i)
    make_rotated_stamp(visit_4, 'fig/system_%s_4.fits'%i)

### 2.2. Generating Coadd Stamps

We'll also need a stamp that "averages" each system over all the time indices to simulate what that system would look like in a `deep_coadd` image. Again `lib/stamp.py` defines a method `get_coadd_stamp` that takes the index of our SL-AGN system and produces a coadd stamp. Recall since `deep_coadd` images are aligned to the ra,dec grid, we don't need to rotate these to match any wcs.

In [ ]:
for i in ([1,2,4,5]):
    get_coadd_stamp(i)

## 3. Saving Our Progress

Now we've successfully obtained all the images and stamps we'll need for our source detection and analysis! Finally, let's save everything to a folder `catalog/` so that we can obtain it all in later notebooks without having to re-run all the above code. We'll do this using a method called `save_pickle`.

In [ ]:
save_pickle("visit_0", visit_0)
save_pickle("visit_1", visit_1)
save_pickle("visit_2", visit_2)
save_pickle("visit_3", visit_3)
save_pickle("visit_4", visit_4)
save_pickle("coadd", coadd)

save_pickle("visit_data", visit_data)